In [1]:
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://archive.apache.org/dist/spark/spark-2.3.1/spark-2.3.1-bin-hadoop2.7.tgz
!tar xf spark-2.3.1-bin-hadoop2.7.tgz
!pip install -q findspark

import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.3.1-bin-hadoop2.7"

import findspark
findspark.init()

import pyspark

# SparkContext
from pyspark import SparkConf, SparkContext
conf = SparkConf().setMaster("local").setAppName("My App")
sc = SparkContext(conf=conf)

# SparkSession
from pyspark.sql import SparkSession

#spark = SparkSession.builder.getOrCreate() 
spark = SparkSession.builder.appName("My App").getOrCreate()
spark

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:9 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:10 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [66.2 kB]
Get:11 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:13 http://ppa.launchpad.net/cran/

In [30]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Learning PySpark
### Chapter 4: DataFrames
This notebook contains sample code from Chapter 4 of [Learning PySpark]() focusing on PySpark and DataFrames.

### Generate your own DataFrame
ファイルシステムにアクセスするのではなく、データを生成してDataFrameを作ってみましょう。今回は、まず`stringRDD`というRDDを作成し、`spark.read.json`を使ってstringJSONRDDを読み込む際にDataFrameに変換します。

In [2]:
# Generate our own JSON data 
#   This way we don't have to access the file system yet.
stringJSONRDD = sc.parallelize((""" 
  { "id": "123",
    "name": "Katie",
    "age": 19,
    "eyeColor": "brown"
  }""",
   """{
    "id": "234",
    "name": "Michael",
    "age": 22,
    "eyeColor": "green"
  }""", 
  """{
    "id": "345",
    "name": "Simone",
    "age": 23,
    "eyeColor": "blue"
  }""")
)

In [3]:
# Create DataFrame
swimmersJSON = spark.read.json(stringJSONRDD)

In [4]:
# Create temporary table
swimmersJSON.createOrReplaceTempView("swimmersJSON")

In [5]:
# DataFrame API
swimmersJSON.show()

+---+--------+---+-------+
|age|eyeColor| id|   name|
+---+--------+---+-------+
| 19|   brown|123|  Katie|
| 22|   green|234|Michael|
| 23|    blue|345| Simone|
+---+--------+---+-------+



In [6]:
# SQL Query
spark.sql("select * from swimmersJSON").collect()

[Row(age=19, eyeColor='brown', id='123', name='Katie'),
 Row(age=22, eyeColor='green', id='234', name='Michael'),
 Row(age=23, eyeColor='blue', id='345', name='Simone')]

In [7]:
'''
%sql 
-- Query Data
select * from swimmersJSON
'''

'\n%sql \n-- Query Data\nselect * from swimmersJSON\n'

#### Inferring the Schema Using Reflection
リフレクションを利用してスキーマを推定する方法

In [8]:
# Print the schema
swimmersJSON.printSchema()

root
 |-- age: long (nullable = true)
 |-- eyeColor: string (nullable = true)
 |-- id: string (nullable = true)
 |-- name: string (nullable = true)



Sparkはスキーマを推測することができたことに注目してください（.printSchemaを使ってスキーマを確認した場合）。

しかし、プログラムでスキーマを指定したい場合はどうでしょうか？

#### Programmatically Specifying the Schema
In this case, let's specify the schema for a `CSV` text file.

In [9]:
from pyspark.sql.types import *

# Generate our own CSV data 
#   This way we don't have to access the file system yet.
stringCSVRDD = sc.parallelize([(123, 'Katie', 19, 'brown'), (234, 'Michael', 22, 'green'), (345, 'Simone', 23, 'blue')])

# The schema is encoded in a string, using StructType we define the schema using various pyspark.sql.types
schemaString = "id name age eyeColor"
schema = StructType([
    StructField("id", LongType(), True),    
    StructField("name", StringType(), True),
    StructField("age", LongType(), True),
    StructField("eyeColor", StringType(), True)
])

# Apply the schema to the RDD and Create DataFrame
swimmers = spark.createDataFrame(stringCSVRDD, schema)

# Creates a temporary view using the DataFrame
swimmers.createOrReplaceTempView("swimmers")

In [10]:
# Print the schema
#   Notice that we have redefined id as Long (instead of String)
swimmers.printSchema()

root
 |-- id: long (nullable = true)
 |-- name: string (nullable = true)
 |-- age: long (nullable = true)
 |-- eyeColor: string (nullable = true)



In [12]:
'''
%sql 
-- Query the data
select * from swimmers
'''

'\n%sql \n-- Query the data\nselect * from swimmers\n'

上記からわかるように，Sparkエンジンがリフレクションによってスキーマを推測するのではなく，プログラムによってスキーマを適用することができます。

Additional Resources include:
* [PySpark API Reference](https://spark.apache.org/docs/2.0.0/api/python/pyspark.sql.html)
* [Spark SQL, DataFrames, and Datasets Guide](https://spark.apache.org/docs/latest/sql-programming-guide.html#programmatically-specifying-the-schema): This is in reference to Programmatically Specifying the Schema using a `CSV` file.

####|| SparkSession
 
`sqlContext.read...`ではなく、`spark.read....` になっていることに注目してください。これは、Spark 2.0の一部として、`HiveContext`、`SQLContext`、`StreamingContext`、`SparkContext`がSpark Session `spark`に統合されたためです。
* データ読み込みのエントリーポイント
* メタデータの活用
* Configuration
* クラスターのリソース管理

For more information, please refer to [How to use SparkSession in Apache Spark 2.0](http://bit.ly/2br0Fr1) (http://bit.ly/2br0Fr1).

### Querying with SQL
DataFramesでは、Hive Query Language（HiveQL）と互換性のあるSQL言語である`Spark SQL`を使って、クエリを書き始めることができます。

In [13]:
# Execute SQL Query and return the data
spark.sql("select * from swimmers").show()

+---+-------+---+--------+
| id|   name|age|eyeColor|
+---+-------+---+--------+
|123|  Katie| 19|   brown|
|234|Michael| 22|   green|
|345| Simone| 23|    blue|
+---+-------+---+--------+



Let's get the row count:

In [14]:
# Get count of rows in SQL
spark.sql("select count(1) from swimmers").show()

+--------+
|count(1)|
+--------+
|       3|
+--------+



なお、%sqlはDatabricksノートブックのノートブック・セル内で使用することができます。

In [15]:
'''
%sql 
-- Query all data
select * from swimmers
'''

'\n%sql \n-- Query all data\nselect * from swimmers\n'

In [16]:
# Query id and age for swimmers with age = 22 via DataFrame API
swimmers.select("id", "age").filter("age = 22").show()

+---+---+
| id|age|
+---+---+
|234| 22|
+---+---+



In [17]:
# Query id and age for swimmers with age = 22 via DataFrame API in another way
swimmers.select(swimmers.id, swimmers.age).filter(swimmers.age == 22).show()


+---+---+
| id|age|
+---+---+
|234| 22|
+---+---+



In [18]:
# Query id and age for swimmers with age = 22 in SQL
spark.sql("select id, age from swimmers where age = 22").show()

+---+---+
| id|age|
+---+---+
|234| 22|
+---+---+



In [21]:
'''
%sql 
-- Query id and age for swimmers with age = 22
select id, age from swimmers where age = 22
'''

'\n%sql \n-- Query id and age for swimmers with age = 22\nselect id, age from swimmers where age = 22\n'

In [22]:
# Query name and eye color for swimmers with eye color starting with the letter 'b'
spark.sql("select name, eyeColor from swimmers where eyeColor like 'b%'").show()

+------+--------+
|  name|eyeColor|
+------+--------+
| Katie|   brown|
|Simone|    blue|
+------+--------+



In [24]:
'''
%sql 
-- Query name and eye color for swimmers with eye color starting with the letter 'b'
select name, eyeColor from swimmers where eyeColor like 'b%
'''

"\n%sql \n-- Query name and eye color for swimmers with eye color starting with the letter 'b'\nselect name, eyeColor from swimmers where eyeColor like 'b%\n"

### Querying with the DataFrame API
DataFramesでは、DataFrame APIを使用してクエリを書き始めることができます。

In [25]:
# Show the values 
swimmers.show()

+---+-------+---+--------+
| id|   name|age|eyeColor|
+---+-------+---+--------+
|123|  Katie| 19|   brown|
|234|Michael| 22|   green|
|345| Simone| 23|    blue|
+---+-------+---+--------+



In [26]:
# Using Databricks `display` command to view the data easier
display(swimmers)

DataFrame[id: bigint, name: string, age: bigint, eyeColor: string]

In [27]:
# Get count of rows
swimmers.count()

3

In [28]:
# Get the id, age where age = 22
swimmers.select("id", "age").filter("age = 22").show()

+---+---+
| id|age|
+---+---+
|234| 22|
+---+---+



In [29]:
# Get the name, eyeColor where eyeColor like 'b%'
swimmers.select("name", "eyeColor").filter("eyeColor like 'b%'").show()

+------+--------+
|  name|eyeColor|
+------+--------+
| Katie|   brown|
|Simone|    blue|
+------+--------+



## On-Time Flight Performance
フライトの出発遅延を州と都市別に検索するには、出発遅延を結合し、空港コード（州と都市を識別するため）に結合します。

### DataFrame Queries
DataFramesを使ってフライトパフォーマンスを実行してみましょう。まず、ソースデータセットからDataFramesを構築します。

In [33]:
# Set File Paths
# flightPerfFilePath = "/databricks-datasets/flights/departuredelays.csv"
# airportsFilePath = "/databricks-datasets/flights/airport-codes-na.txt"
flightPerfFilePath = "/content/drive/MyDrive/Colab Notebooks/data/departuredelays.csv"
airportsFilePath = "/content/drive/MyDrive/Colab Notebooks/data/airport-codes-na.txt"

# Obtain Airports dataset
airports = spark.read.csv(airportsFilePath, header='true', inferSchema='true', sep='\t')
airports.createOrReplaceTempView("airports")

# Obtain Departure Delays dataset
flightPerf = spark.read.csv(flightPerfFilePath, header='true')
flightPerf.createOrReplaceTempView("FlightPerformance")

# Cache the Departure Delays dataset 
flightPerf.cache()

DataFrame[date: string, delay: string, distance: string, origin: string, destination: string]

In [34]:
# Query Sum of Flight Delays by City and Origin Code (for Washington State)
spark.sql("select a.City, f.origin, sum(f.delay) as Delays from FlightPerformance f join airports a on a.IATA = f.origin where a.State = 'WA' group by a.City, f.origin order by sum(f.delay) desc").show()

+-------+------+--------+
|   City|origin|  Delays|
+-------+------+--------+
|Seattle|   SEA|159086.0|
|Spokane|   GEG| 12404.0|
|  Pasco|   PSC|   949.0|
+-------+------+--------+



In [36]:
'''%sql
-- Query Sum of Flight Delays by City and Origin Code (for Washington State)
select a.City, f.origin, sum(f.delay) as Delays
  from FlightPerformance f
    join airports a
      on a.IATA = f.origin
 where a.State = 'WA'
 group by a.City, f.origin
 order by sum(f.delay) desc
 '''

"%sql\n-- Query Sum of Flight Delays by City and Origin Code (for Washington State)\nselect a.City, f.origin, sum(f.delay) as Delays\n  from FlightPerformance f\n    join airports a\n      on a.IATA = f.origin\n where a.State = 'WA'\n group by a.City, f.origin\n order by sum(f.delay) desc\n "

In [37]:
# Query Sum of Flight Delays by State (for the US)
spark.sql("select a.State, sum(f.delay) as Delays from FlightPerformance f join airports a on a.IATA = f.origin where a.Country = 'USA' group by a.State ").show()

+-----+---------+
|State|   Delays|
+-----+---------+
|   SC|  80666.0|
|   AZ| 401793.0|
|   LA| 199136.0|
|   MN| 256811.0|
|   NJ| 452791.0|
|   OR| 109333.0|
|   VA|  98016.0|
| null| 397237.0|
|   RI|  30760.0|
|   WY|  15365.0|
|   KY|  61156.0|
|   NH|  20474.0|
|   MI| 366486.0|
|   NV| 474208.0|
|   WI| 152311.0|
|   ID|  22932.0|
|   CA|1891919.0|
|   CT|  54662.0|
|   NE|  59376.0|
|   MT|  19271.0|
+-----+---------+
only showing top 20 rows



In [39]:
'''
%sql
-- Query Sum of Flight Delays by State (for the US)
select a.State, sum(f.delay) as Delays
  from FlightPerformance f
    join airports a
      on a.IATA = f.origin
 where a.Country = 'USA'
 group by a.State '''

"\n%sql\n-- Query Sum of Flight Delays by State (for the US)\nselect a.State, sum(f.delay) as Delays\n  from FlightPerformance f\n    join airports a\n      on a.IATA = f.origin\n where a.Country = 'USA'\n group by a.State "

In [40]:
'''%sql
-- Query Sum of Flight Delays by State (for the US)
select a.State, sum(f.delay) as Delays
  from FlightPerformance f
    join airports a
      on a.IATA = f.origin
 where a.Country = 'USA'
 group by a.State '''

"%sql\n-- Query Sum of Flight Delays by State (for the US)\nselect a.State, sum(f.delay) as Delays\n  from FlightPerformance f\n    join airports a\n      on a.IATA = f.origin\n where a.Country = 'USA'\n group by a.State "

For more information, please refer to:
* [Spark SQL, DataFrames and Datasets Guide](http://spark.apache.org/docs/latest/sql-programming-guide.html#sql)
* [PySpark SQL Module: DataFrame](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrame)
* [PySpark SQL Functions Module](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#module-pyspark.sql.functions)